In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
master_path = 'c:/DataCleanup/Data/'
master_file = 'Data Cleanup Tracking Sheet.csv'
master_sheet = 'Full Tracking List'
#try:
file_df = pd.read_csv(master_path + master_file, low_memory=False, index_col='ProfileID_M')
#except:
#    OverflowError as of:
file_df

,FirstName_M,LastName_M,Phone_M,email_M,Class_M,Specialty_M,Address_M,City_M,State_M,Zip_M,...,AlertMediaUpdated,GroupsUpdated,Removed_AM,AM_By:,MobilizeUpdated,Removed_Mob,MobilizeBy:,BackendUpdated,BE_By:,Completed
ProfileID_M,,,,,,,,,,,,,,,,,,,,,
19908,Fatimah,Abdullah,000-000-0000,fabdullah_3@yahoo.com,Certified Nursing Assistant,Geriatrics,30760 eden allen road,Eden,MD,21822,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19909,Julie,Szabo,413-770-7671,szabo.julie@yahoo.com,Licensed Practical Nurse,Geriatrics; Clinic,12 von nida ave,Pittsfield,MA,1201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19963,David,Barsamian,000-000-0000,noemail@favoritestaffing.com,Registered Nurse,Medical/Surgical; Telemetry,380 east center st,Manchester,CT,6040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19974,Tracy,Seelbaugh,724-372-4466; 724-712-4688,tlseelbaugh@gmail.com,Registered Nurse; Certified Nursing Assistant,Geriatrics,40 Enclave Dr,Grove City,PA,16127,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19991,Karen,Wlostoski,413-773-7298,sasha65@comcast.net,Registered Nurse,Medical/Surgical; Geriatrics; Telemetry; In...,PO Box 532,Greenfield,MA,1302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051730,Sashauna,Brown,860-655-4979,Sashaunabrown30@gmail.com,Registered Nurse,Charge,NaN,NaN,NaN,6053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051731,Chloe,Iverson,609-361-4626,chloeiverson73@yahoo.com,Patient Care Assistant/Tech,Intensive Care Unit,NaN,NaN,NaN,8050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051732,Matthew,Hennessy,516-524-5514,hennessymatthew56@gmail.com,Paramedic,Emergency Management,911 Clydesdale Ct,Wilmington,NC,28411,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
delete_df = file_df.loc[file_df['AM Delete'] == 'Delete',['FirstName_M', 'LastName_M', 'email_M', 'AM Delete']]

In [4]:
delete_df


,FirstName_M,LastName_M,email_M,AM Delete
ProfileID_M,,,,
26777,Joseph,Battig,michaelbattig@gmail.com,Delete
33846,Reginald,Hills,thills2023@gmail.com,Delete
39383,Tori,Marsh,torisfarm@aol.com,Delete
39715,Giovanna,Barone,joann623@yahoo.com,Delete
41670,Robert,Haberle,misouritraveller@aol.com,Delete
...,...,...,...,...
1004000,Christopher,Bartley,christopher.c.bartley@gmail.com,Delete
1004510,.,.,none@none.com,Delete
1029174,Sara,Barber,saraebarb@gmail.com,Delete


DNR List?